In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\robot\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [24]:
import numpy as np
import json
import glob
from pathlib import Path
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
from pyLDAvis.gensim import prepare

c:\Users\robot\Documents\Repos\ResearchRAG\.venv\Lib\site-packages\spacy\cli\_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
c:\Users\robot\Documents\Repos\ResearchRAG\.venv\Lib\site-packages\weasel\util\config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


In [9]:
stopwords = stopwords.words("english")
stopwords

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [14]:
# load data

data_dir = Path("data")
# Load data
old_df = pd.read_csv(Path("..") / data_dir / "kaggle_arxiv_dataset" / "dataset.csv")
# new_df = pd.read_csv(data_dir / "arxiv_dataset" / "")

old_df

,Unnamed: 0,authors,journal_entry,pdf_url,publish_date,title,abstract
0,0,[arxiv.Result.Author('M. L. Ginsberg')],http://arxiv.org/abs/cs/9308101v1,http://arxiv.org/pdf/cs/9308101v1,1993-08-01 00:00:00+00:00,Dynamic Backtracking,Because of their occasional need to return to ...
1,1,[arxiv.Result.Author('M. P. Wellman')],http://arxiv.org/abs/cs/9308102v1,http://arxiv.org/pdf/cs/9308102v1,1993-08-01 00:00:00+00:00,A Market-Oriented Programming Environment and ...,Market price systems constitute a well-underst...
2,2,"[arxiv.Result.Author('I. P. Gent'), arxiv.Resu...",http://arxiv.org/abs/cs/9309101v1,http://arxiv.org/pdf/cs/9309101v1,1993-09-01 00:00:00+00:00,An Empirical Analysis of Search in GSAT,We describe an extensive study of search in GS...
3,3,"[arxiv.Result.Author('F. Bergadano'), arxiv.Re...",http://arxiv.org/abs/cs/9311101v1,http://arxiv.org/pdf/cs/9311101v1,1993-11-01 00:00:00+00:00,The Difficulties of Learning Logic Programs wi...,As real logic programmers normally use cut (!)...
4,4,"[arxiv.Result.Author('J. C. Schlimmer'), arxiv...",http://arxiv.org/abs/cs/9311102v1,http://arxiv.org/pdf/cs/9311102v1,1993-11-01 00:00:00+00:00,Software Agents: Completing Patterns and Const...,To support the goal of allowing users to recor...
...,...,...,...,...,...,...,...
9995,9995,"[arxiv.Result.Author('Maxime Lefrançois'), arx...",http://arxiv.org/abs/1204.5316v1,http://arxiv.org/pdf/1204.5316v1,2012-04-24 09:13:59+00:00,ILexicOn: toward an ECD-compliant interlingual...,We are interested in bridging the world of nat...
9996,9996,"[arxiv.Result.Author('Yoshiharu Maeno'), arxiv...",http://arxiv.org/abs/1204.5661v2,http://arxiv.org/pdf/1204.5661v2,2012-04-25 14:07:22+00:00,Transmission of distress in a bank credit network,The European sovereign debt crisis has impaire...
9997,9997,"[arxiv.Result.Author('C. Widanapathirana'), ar...",http://arxiv.org/abs/1204.5805v1,http://arxiv.org/pdf/1204.5805v1,2012-04-26 01:49:57+00:00,Intelligent Automated Diagnosis of Client Devi...,We present an automated solution for rapid dia...
9998,9998,"[arxiv.Result.Author('Paolo Liberatore'), arxi...",http://arxiv.org/abs/1204.5859v3,http://arxiv.org/pdf/1204.5859v3,2012-04-26 08:18:38+00:00,On the Complexity of Finding Second-Best Abduc...,While looking for abductive explanations of a ...


In [31]:
# Combine title and abstract relevant columns
def combine_text(df) -> list:
    return (df["title"] + " " + df["abstract"]).fillna("").tolist()

combined_old_data: list[str] = combine_text(old_df)

In [26]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [32]:
def lemmatization(texts:list, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

    texts_out = [" ".join([token.lemma_ for token in nlp(text) if token.pos_ in allowed_postags]) for text in texts ]
    return texts_out

lemmatized_texts = lemmatization(combined_old_data)
print(lemmatized_texts[0][0:90])

dynamic occasional need return shallow point search tree exist backtrack method sometimes 


In [33]:
def gen_words(texts):
    return [gensim.utils.simple_preprocess(text, deacc=True) for text in texts]

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['dynamic', 'occasional', 'need', 'return', 'shallow', 'point', 'search', 'tree', 'exist', 'backtrack', 'method', 'sometimes', 'erase', 'meaningful', 'progress', 'solve', 'search', 'problem', 'paper', 'present']


In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]

print(corpus[0][0:20])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1)]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=30,
                                            random_state=0,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha="auto",
                                            )